## 1.Simple Chain

In [1]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq


load_dotenv()
groq_api=os.getenv('groq_api')
llm=ChatGroq(model='gemma2-9b-it',api_key=groq_api)


In [4]:
prompt = PromptTemplate(
    template='Generate 5 interesting facts about {topic}',
    input_variables=['topic']
)

parser=StrOutputParser()

chain=prompt|llm|parser

output=chain.invoke({'topic':'Cricket'})

In [5]:
print(output)

Here are 5 interesting facts about cricket:

1. **Cricket originated in England, but is now most popular in Asia and Australia:** While it began in villages across England as early as the 16th century, cricket's global popularity surged in the 20th century thanks to India, Pakistan, Australia, and other nations. Today, it's a truly international sport.

2. **The world's largest cricket stadium is in India:** The Narendra Modi Stadium in Ahmedabad, India, boasts a capacity of over 132,000 spectators. That’s larger than the population of many small towns!

3. **There are more formats of cricket than just Test Matches:** Alongside the traditional five-day Test matches, there's One Day International (ODI) cricket, which lasts for a single day, and Twenty20 (T20) cricket, a fast-paced version that completes in around three hours.

4. **"Duck" isn't just a bird, it's a cricketing term:** If a batsman scores zero runs and gets out in the same innings, they are said to have been "duck."  

5. 

## 2. Sequential

In [7]:
prompt=PromptTemplate(
    template="Generate a 4 point summary from this text {text}",
    input_variables=['text']
)

chain=prompt|llm|parser
chain.invoke({'text':output})

'Here\'s a 4-point summary of the text:\n\n* **Global Phenomenon:** Cricket originated in England but gained massive popularity in Asia and Australia, making it a truly international sport.\n* **Stadium Spectacle:** The Narendra Modi Stadium in India holds the record for the world\'s largest cricket stadium with a capacity exceeding 132,000 spectators.\n* **Variety of Formats:**  Cricket encompasses various formats beyond the traditional Test Matches, including One Day Internationals (ODIs) and fast-paced Twenty20 (T20) matches.\n* **Cricket Terminology:**  "Duck" in cricket refers to a batsman scoring zero runs and getting out within the same innings. \n\n\n\n'

## 3. Parallel

In [15]:
from langchain.schema.runnable import RunnableParallel


prompt1=PromptTemplate(
    template="Generate a short simple Note on the text of {text}",
    input_variables=['text']
)
prompt2 = PromptTemplate(
    template='Generate 5 short question answers from the following text \n {text}',
    input_variables=['text']
)

prompt3 = PromptTemplate(
    template='Merge the provided notes and quiz into a single document \n notes -> {notes} and quiz -> {quiz}',
    input_variables=['notes', 'quiz']
)

In [16]:

paralle_chain=RunnableParallel(
    {
        'notes':prompt1|llm|parser,
        'quiz':prompt2|llm|parser
        
        
    }
)
merge_chain=prompt3|llm|parser
final_chain=paralle_chain|merge_chain
text = """
Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

The advantages of support vector machines are:

Effective in high dimensional spaces.

Still effective in cases where number of dimensions is greater than the number of samples.

Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.

Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

The disadvantages of support vector machines include:

If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.

SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation (see Scores and probabilities, below).

The support vector machines in scikit-learn support both dense (numpy.ndarray and convertible to that by numpy.asarray) and sparse (any scipy.sparse) sample vectors as input. However, to use an SVM to make predictions for sparse data, it must have been fit on such data. For optimal performance, use C-ordered numpy.ndarray (dense) or scipy.sparse.csr_matrix (sparse) with dtype=float64.
"""
response=final_chain.invoke({'text':text})

In [17]:
response

"## Support Vector Machines (SVMs): A Quick Overview\n\n**What they are:** SVMs are supervised learning algorithms used for classification, regression, and outlier detection.\n\n**Advantages:**\n\n* **Effective in high dimensions:**  Work well even when the number of features exceeds the number of samples.\n* **Memory efficient:**  Only use a subset of training data (support vectors) for decision making.\n* **Versatile:**  Different kernel functions can be used to customize the decision boundary.\n\n**Disadvantages:**\n\n* **Overfitting risk:**  Careful selection of kernel functions and regularization is crucial when features outnumber samples.\n* **No direct probability estimates:**  Probability calculations require expensive cross-validation.\n\n**Input Data:**\n\n* Can handle both dense (numpy arrays) and sparse (scipy sparse) data.\n* **Important:**  Must train on the same data type used for prediction.\n\n**Optimal Performance:**\n\n* Use C-ordered numpy arrays (dense) or scipy.sp

In [21]:
final_chain.get_graph().print_ascii()

          +---------------------------+            
          | Parallel<notes,quiz>Input |            
          +---------------------------+            
                ***             ***                
              **                   **              
            **                       **            
+----------------+              +----------------+ 
| PromptTemplate |              | PromptTemplate | 
+----------------+              +----------------+ 
          *                             *          
          *                             *          
          *                             *          
    +----------+                  +----------+     
    | ChatGroq |                  | ChatGroq |     
    +----------+                  +----------+     
          *                             *          
          *                             *          
          *                             *          
+-----------------+            +-----------------+ 
| StrOutputP

## conditional chain

In [30]:
from langchain.schema.runnable import RunnableParallel,RunnableBranch,RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from typing import Literal
from pydantic import BaseModel,Field


class Feedback(BaseModel):

    sentiment: Literal['positive', 'negative'] = Field(description='Give the sentiment of the feedback')

parser2 = PydanticOutputParser(pydantic_object=Feedback)

prompt1 = PromptTemplate(
    template='Classify the sentiment of the following feedback text into postive or negative \n {feedback} \n {format_instruction}',
    input_variables=['feedback'],
    partial_variables={'format_instruction':parser2.get_format_instructions()}
)

classifier_chain=prompt1|llm|parser2

prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)

In [31]:
branch_chain=RunnableBranch(
   (lambda x:x.sentiment=='positive',prompt2|llm|parser),
   (lambda x:x.sentiment=='negative',prompt3|llm|parser),
   RunnableLambda(lambda x:"could not find sentiment")
)
chain = classifier_chain | branch_chain

print(chain.invoke({'feedback': 'This is a beautiful phone'}))



Here are a few appropriate responses to positive feedback, depending on the context:

**Formal:**

* "Thank you for your kind words. I'm glad you found [what you liked] helpful/enjoyable."
* "We appreciate your positive feedback. We strive to provide excellent service/products and are pleased to hear we met your expectations."

**Informal:**

* "Thanks so much! I'm so happy you liked it."
* "That means a lot to me! Thanks for the feedback."
* "Awesome! I'm glad it worked out well for you."

**Adding a personal touch:**

* "Thank you! I put a lot of effort into [what you liked] and it's great to know it made a difference."
* "I'm thrilled you enjoyed [what you liked].  [Mention something specific about their feedback]."


No matter which response you choose, make sure it is sincere and reflects your appreciation for the positive feedback. 



In [27]:
chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
      +----------+       
      | ChatGroq |       
      +----------+       
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |        
       +--------+        
            *            
            *            
            *            
    +--------------+     
    | BranchOutput |     
    +--------------+     
